# HOME ASSIGNMENT #5: BIGQUERY SQL & PYTHON PANDAS

**Mục đích của bài Assignment**
> *  Upload Data từ Google Spreadsheet lên BigQuery và thao tác data với SQL
*  Thao tác với data bằng Pandas
*   `[Optional]` Data Visualization on Python

**Các kiến thức áp dụng**
* Slack API, JSON to DataFrame
* Google Spreadsheet to BigQuery
* SQL
* Python Pandas

**Lời Khuyên**
* Đây là bài tập nhẹ nhàng hơn các tuần trước
* Các bạn dành thời gian ôn lại và xâu chuỗi kiến thức

# TODO 1: BIGQUERY & SQL
* Làm theo Lab của Week 5: [HERE](https://anhdang.gitbook.io/datacracy/atom/5-sql-and-database-design/5.2-lab-upload-data-tu-google-sheet-to-bigquery)

In [1]:
# Import Library
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os
import numpy as np

In [2]:
# (1) Code để pull Slack Data
with open('env_variable.json', 'r') as j:
    json_data = json.load(j)

In [3]:
# Code để upload data lên Gsheet
# 1. LIST OF SLACK MEMBERS 

## Pull list of member as JSON
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(json_data['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

## Convert to CSV
user_dict = {'user_id':[],'name':[],'display_name':[],'real_name':[],'title':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])

user_df = pd.DataFrame(user_dict) 
user_df = user_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)

In [4]:
# 2. LIST OF SLACK CHANNELS
endpoint2 = "https://slack.com/api/conversations.list"
data = {'types': 'public_channel,private_channel'} # -> CHECK: API Docs https://api.slack.com/methods/conversations.list/test
response_json = requests.post(endpoint2, headers=headers, data=data).json() 
channel_dat = response_json['channels']

channel_dict = {'channel_id':[], 'channel_name':[], 'is_channel':[],'creator':[],'created_at':[],'topics':[],'purpose':[],'num_members':[]}
for i in range(len(channel_dat)):
  channel_dict['channel_id'].append(channel_dat[i]['id'])
  channel_dict['channel_name'].append(channel_dat[i]['name'])
  channel_dict['is_channel'].append(channel_dat[i]['is_channel'])
  channel_dict['creator'].append(channel_dat[i]['creator'])
  channel_dict['created_at'].append(dt.fromtimestamp(float(channel_dat[i]['created'])))
  channel_dict['topics'].append(channel_dat[i]['topic']['value'])
  channel_dict['purpose'].append(channel_dat[i]['purpose']['value'])
  channel_dict['num_members'].append(channel_dat[i]['num_members'])

channel_df = pd.DataFrame(channel_dict) 
channel_df = channel_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)

In [5]:
# 3. Message Data
endpoint3 = "https://slack.com/api/conversations.history"
msg_dict = {'channel_id':[],'msg_id':[], 'msg_ts':[], 'user_id':[], 'latest_reply':[],'reply_user_count':[],'reply_users':[],'github_link':[]}
for channel_id, channel_name in zip(channel_df['channel_id'], channel_df['channel_name']):
  print('Channel ID: {} - Channel Name: {}'.format(channel_id, channel_name))
  try:
    data = {"channel": channel_id} 
    response_json = requests.post(endpoint3, data=data, headers=headers).json()
    msg_ls = response_json['messages']
    for i in range(len(msg_ls)):
      if 'client_msg_id' in msg_ls[i].keys():
        msg_dict['channel_id'].append(channel_id)
        msg_dict['msg_id'].append(msg_ls[i]['client_msg_id'])
        msg_dict['msg_ts'].append(dt.fromtimestamp(float(msg_ls[i]['ts'])))
        msg_dict['latest_reply'].append(dt.fromtimestamp(float(msg_ls[i]['latest_reply'] if 'latest_reply' in msg_ls[i].keys() else 0))) ## -> No reply: 1970-01-01
        msg_dict['user_id'].append(msg_ls[i]['user'])
        msg_dict['reply_user_count'].append(msg_ls[i]['reply_users_count'] if 'reply_users_count' in msg_ls[i].keys() else 0)
        msg_dict['reply_users'].append(msg_ls[i]['reply_users'] if 'reply_users' in msg_ls[i].keys() else 0) 
        ## -> Censor message contains tokens
        text = msg_ls[i]['text']
        github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text)
        msg_dict['github_link'].append(github_link[0] if len(github_link) > 0 else np.nan)
  except:
    print('====> '+ str(response_json))
  
msg_df = pd.DataFrame(msg_dict)
msg_df = msg_df.replace('', np.nan) # -> replace khoảng trắng bằng giá trị NULL (nan)

Channel ID: C01B4PVGLVB - Channel Name: general
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01BYH7JHB5 - Channel Name: contents
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01CAMNCJJV - Channel Name: branding-design
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C01U6P7LZ8F - Channel Name: atom-assignment1
Channel ID: C01UL6K1C7L - Channel Name: atom-week1
Channel ID: C01ULCHGN75 - Channel Name: atom-general
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C020VMT58JK - Channel Name: topics-data-analytics
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C0213MNH9L6 - Channel Name: topics-python
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C0213N56M2A - Channel Name: topics-materials
====> {'ok': False, 'error': 'not_in_channel'}
Channel ID: C021FSDN7LJ - Channel Name: atom-assignment2
Channel ID: C021KLB0DSB - Channel Name: discuss-group3
Channel ID: C021KLB90GP - Channel Name: discuss-group4
Channel ID: C0220

In [6]:
# 4. DTC_groups data
dtc_groups = pd.read_csv('datacracy_groups.csv')

In [7]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'api-slack-huynh-tan-phuoc-73998bbc82a9.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


In [9]:
# ACCES GOOGLE SHEET
spreadsheet_key = '1sX-0rAKZYM1cQyH7HWzTPxoPD0qo-zJd_PrgCPoRRBs' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
df = [user_df, channel_df, msg_df, dtc_groups]
for sheet_index_no in range(len(df)):
    worksheet = sh.get_worksheet(sheet etc. x_no]) #-> Upload user_df vào Sheet 0, 1 - etc

### Link tới Gsheet: https://docs.google.com/spreadsheets/d/1sX-0rAKZYM1cQyH7HWzTPxoPD0qo-zJd_PrgCPoRRBs/edit#gid=0

![query](../img/query.PNG)

In [3]:
# Copy các link Kaggle sau:
## 1. Link tới Kaggle Account của bạn -----> https://www.kaggle.com/interphuoc0101
## 2. Link tới các bài tập
## Pandas 1: ---> https://www.kaggle.com/interphuoc0101/exercise-creating-reading-and-writing
## Pandas 2: ---> https://www.kaggle.com/interphuoc0101/exercise-indexing-selecting-assigning
## Pandas 3: ---> https://www.kaggle.com/interphuoc0101/exercise-summary-functions-and-maps
## Pandas 4: ---> https://www.kaggle.com/interphuoc0101/exercise-grouping-and-sorting
## Pandas 5: ---> https://www.kaggle.com/interphuoc0101/exercise-data-types-and-missing-values
## Pandas 6: ---> https://www.kaggle.com/interphuoc0101/exercise-renaming-and-combining

# TODO 3: Python Pandas 
Trở lại TODO7 [Assignment#4](https://github.com/anhdanggit/atom-assignments/blob/main/assignment_4/home_assignment_4.ipynb) (nếu bạn chưa hoàn thành)

In [19]:
# 2. Learner nào nộp bài trễ nhất trong Assignment 1, 2, 3?
# Dùng Python pandas
step_1 = pd.merge(msg_df, channel_df, how='left', on='channel_id')
step_2 = pd.merge(step_1, user_df, how='left', on='user_id')
step_3 = pd.merge(step_2, dtc_groups, how='left', on='name')
step_4 = step_3[step_3['github_link'].notnull()]
step_5 = step_4.loc[step_4['channel_name'].str.startswith('atom-assignment')]
step_6 = step_5.loc[~step_5['channel_name'].str.startswith('atom-assignment4')]
step_7 = step_6.loc[step_6['DataCracy_role'].str.startswith('Learner')]
step_8 = step_7.groupby(['channel_id', 'channel_name'])['msg_ts'].max().reset_index()
step_9 = pd.merge(step_8, msg_df, how='left', on='msg_ts')
step_10 = pd.merge(step_9, user_df, how='left', on='user_id')
step_11 = pd.merge(step_10, dtc_groups, how='left', on='name')
final = step_11.drop(columns=['msg_id', 'latest_reply', 'channel_id_y', 'reply_user_count', 'reply_users', 'display_name', 'real_name', 'title', 'is_bot'])
final

,channel_id_x,channel_name,msg_ts,user_id,github_link,name,DataCracy_role
0,C01U6P7LZ8F,atom-assignment1,2021-05-10 11:51:46.004700,U01UJ9LG5U5,https://github.com/danhpcv/,danhpcv,Learner_Gr3
1,C021FSDN7LJ,atom-assignment2,2021-05-22 12:39:34.015500,U01UTGRMGEQ,https://github.com/nguyenhonghanh2511/,hanh.nguyenhong2511,Learner_Gr4
2,C0227A51SAY,atom-assignment3,2021-05-27 22:30:16.003200,U01USGKQ771,https://github.com/TrangNgo1709/,trang.ngo1709,Learner_Gr1


# TODO 4 (OPTIONAL): Python Data Viz
Coi tutorial và làm nhiều nhất có thể các sets bài tập trên [Kaggle Data Visualization](https://www.kaggle.com/learn/data-visualization)

In [ ]:
# Copy Link tới các bài tập
## DataViz 1: ---> https://www.kaggle.com/interphuoc0101/exercise-hello-seaborn
## DataViz 2: ---> https://www.kaggle.com/interphuoc0101/exercise-line-charts
## DataViz 3: ---> https://www.kaggle.com/interphuoc0101/exercise-bar-charts-and-heatmaps
## DataViz 4: ---> https://www.kaggle.com/interphuoc0101/exercise-scatter-plots
## DataViz 5: ---> https://www.kaggle.com/interphuoc0101/exercise-distributions
## DataViz 6: ---> https://www.kaggle.com/interphuoc0101/exercise-choosing-plot-types-and-custom-styles
## DataViz 7: ---> https://www.kaggle.com/interphuoc0101/exercise-final-project